In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os
import pix2pix
import cv2
import matplotlib.pyplot as plt
from pix2pix import models
import tqdm
from pix2pix import config



In [3]:
gt_path = 'videos/input_video_gt.mp4'
noisy_path = 'videos/input_video_noisy.mp4'
pred_save_path = 'videos/predicted_video_v3.mp4'

device = "cuda" if torch.cuda.is_available() else "cpu"
bs = 256
print (device)

cuda


In [4]:
# state_dict = torch.load('../../models/pix2pix_32_11112021_051302/checkpoints/gen_epoch_1_pix2pix_32_11112021_051302.pt')
# state_dict
state_dict = torch.load('../../models/pix2pix_64_11112021_115558/checkpoints/gen_epoch_2_pix2pix_64_11112021_115558.pt')

In [5]:
model = models.unet(norm_layer=torch.nn.InstanceNorm2d, **config.model.GEN_ARGS)
model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)
model = model.module
model.eval();

In [6]:
class img_dataset(torch.utils.data.Dataset):
    def __init__(self, imgs):
        self.imgs = imgs

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.to_list()
        
        inp = torch.Tensor(self.imgs[idx])
        inp = (inp / (255 / 2)) - 1
        inp = inp.permute((2, 0, 1))
        
        return inp    

In [7]:
%%time

noisy_cap = cv2.VideoCapture(noisy_path)
fps = noisy_cap.get(cv2.CAP_PROP_FPS)
width = int(noisy_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(noisy_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter(pred_save_path, fourcc, fps, (width, height))
    
frame_list = []

while noisy_cap.isOpened():
    ret, frame = noisy_cap.read()
    
    if ret == False:
        break
        
    frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
noisy_cap.release()
img_dataloader = torch.utils.data.DataLoader(img_dataset(frame_list), 
                                             batch_size = 128,
                                             shuffle = False,
                                            num_workers = 1)

model.eval()
preds_list = []

for ip in tqdm.tqdm(img_dataloader, total = len(img_dataloader)):    
    with torch.no_grad():
        ip = ip.to(device)
        preds = model(ip).detach()
            
        preds = ((preds + 1) / 2).permute(0, 2, 3, 1).to('cpu') 
        preds = (preds.numpy() * 255).astype('uint8')
    
    preds_list.extend(list(preds))
    
for pred in preds_list:
    out.write(cv2.cvtColor(pred, cv2.COLOR_RGB2BGR))
    cv2.imshow('asd', cv2.cvtColor(pred, cv2.COLOR_RGB2BGR))
    cv2.waitKey(1)

cv2.destroyAllWindows()
out.release()

100%|██████████| 12/12 [00:13<00:00,  1.13s/it]


CPU times: user 17.8 s, sys: 958 ms, total: 18.7 s
Wall time: 21.6 s
